# Overview
Rather than trying to segment, we start off by making a model that simply tries to identify if any boat shows up in the image. 
For this model we can see roughly how it performs in the compititon by guessing the whole image (as an RLE) if any boat shows up (not a very smart startegy, but might provide some interesting results). 

## Beyond
The model could also be useful as a quick way (low resolution images) to screen through lots of images to see if they are likely to have a boat and if they are then run a much more expensive full-resolution U-Net on that sample

## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations). It is slightly easier to keep track of parallel notebooks with different parameters if they are all at the beginning in a clear (machine readable format, see Kaggling with Kaggle (https://www.kaggle.com/kmader/kaggling-with-kaggle).

In [18]:
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# number of validation images to use
VALID_IMG_COUNT = 1000
# maximum number of training images
MAX_TRAIN_IMAGES = 30000
IMG_SIZE = (512,512) # [(224, 224), (384, 384), (512, 512), (640, 640)]
BATCH_SIZE = 2 # [1, 8, 16, 24]
DROPOUT = 0.5
DENSE_COUNT = 128
LEARN_RATE = 1e-3
RGB_FLIP = 1 # should rgb be flipped when rendering images

In [19]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util.montage import montage2d as montage
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
ship_dir = '/Users/janis/Documents/Python/Airbus/input'
train_image_dir = os.path.join(ship_dir, 'train')
test_image_dir = os.path.join(ship_dir, 'test')
import gc; gc.enable() # memory is tight

In [20]:
masks = pd.read_csv(os.path.join(ship_dir,
                                 'train_ship_segmentations.csv'))
print(masks.shape[0], 'masks found')
print(masks['ImageId'].value_counts().shape[0])
masks['path'] = masks['ImageId'].map(lambda x: os.path.join(train_image_dir, x))
masks.head()

131030 masks found


104070


,ImageId,EncodedPixels,path
0,00003e153.jpg,NaN,/Users/janis/Documents/Python/Airbus/input/tra...
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,/Users/janis/Documents/Python/Airbus/input/tra...
2,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...,/Users/janis/Documents/Python/Airbus/input/tra...
3,00021ddc3.jpg,95225 2 95992 5 96760 7 97527 9 98294 9 99062 ...,/Users/janis/Documents/Python/Airbus/input/tra...
4,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...,/Users/janis/Documents/Python/Airbus/input/tra...


# Split into training and validation groups
We stratify by the number of boats appearing so we have nice balances in each set

In [21]:
from sklearn.model_selection import train_test_split
masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)
unique_img_ids['has_ship_vec'] = unique_img_ids['has_ship'].map(lambda x: [x])
masks.drop(['ships'], axis=1, inplace=True)
train_ids, valid_ids = train_test_split(unique_img_ids, 
                 test_size = 0.3, 
                 stratify = unique_img_ids['ships'])
train_df = pd.merge(masks, train_ids)
valid_df = pd.merge(masks, valid_ids)
print(train_df.shape[0], 'training masks')
print(valid_df.shape[0], 'validation masks')

91709 training masks
39321 validation masks


### Examine Number of Ship Images
Here we examine how often ships appear and replace the ones without any ships with 0

In [22]:
train_df = train_df.sample(min(MAX_TRAIN_IMAGES, train_df.shape[0])) # limit size of training set (otherwise it takes too long)
train_df.head()

,ImageId,EncodedPixels,path,ships,has_ship,has_ship_vec
36591,6609ec824.jpg,334454 4 335216 10 335978 16 336740 23 337502 ...,/Users/janis/Documents/Python/Airbus/input/tra...,2,1.0,[1.0]
16121,2d42828e3.jpg,187914 7 188675 14 189443 14 190211 14 190979 ...,/Users/janis/Documents/Python/Airbus/input/tra...,13,1.0,[1.0]
31387,57e99ea4f.jpg,535395 1 536161 4 536928 5 537694 8 538461 10 ...,/Users/janis/Documents/Python/Airbus/input/tra...,7,1.0,[1.0]
14184,27b091c4f.jpg,NaN,/Users/janis/Documents/Python/Airbus/input/tra...,0,0.0,[0.0]
27678,4db390338.jpg,NaN,/Users/janis/Documents/Python/Airbus/input/tra...,0,0.0,[0.0]


In [23]:
train_df[['ships', 'has_ship']].hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1cef58ec18>,
      dtype=object)

# Augment Data

In [24]:
from keras.preprocessing.image import ImageDataGenerator
dg_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 45, 
                  width_shift_range = 0.1, 
                  height_shift_range = 0.1, 
                  shear_range = 0.01,
                  zoom_range = [0.9, 1.25],  
                  brightness_range = [0.5, 1.5],
                  horizontal_flip = True, 
                  vertical_flip = True,
                  fill_mode = 'reflect',
                   data_format = 'channels_last')
valid_args = dict(fill_mode = 'reflect',
                   data_format = 'channels_last')

core_idg = ImageDataGenerator(**dg_args)
valid_idg = ImageDataGenerator(**valid_args)

In [25]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [28]:
train_gen = flow_from_dataframe(core_idg, train_df, 
                             path_col = 'path',
                            y_col = 'has_ship_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE)

# used a fixed dataset for evaluating the algorithm
valid_x, valid_y = next(flow_from_dataframe(valid_idg, 
                               valid_df, 
                             path_col = 'path',
                            y_col = 'has_ship_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = VALID_IMG_COUNT)) # one big batch
print(valid_x.shape, valid_y.shape)

## Ignore next message from keras, values are replaced anyways


Found 0 images belonging to 0 classes.
Reinserting dataframe: 30000 images
## Ignore next message from keras, values are replaced anyways


Found 0 images belonging to 0 classes.


Reinserting dataframe: 39321 images


(1000, 512, 512, 3) (1000, 1)


In [30]:
t_x, t_y = next(train_gen)
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
ax1.imshow(montage_rgb((t_x-t_x.min())/(t_x.max()-t_x.min()))[:, :, ::RGB_FLIP], cmap='gray')
ax1.set_title('images')
ax2.plot(t_y)
ax2.set_title('ships')

x (2, 512, 512, 3) float32 0.0 255.0
y (2, 1) float64 0.0 1.0


Text(0.5,1,'ships')

# Build a Model
We build the pre-trained top model and then use a global-max-pooling (we are trying to detect any ship in the image and thus max is better suited than averaging (which would tend to favor larger ships to smaller ones). 

## Setup the Subsequent Layers
Here we setup the rest of the model which we will actually be training

In [32]:
from keras import models, layers
from keras.optimizers import Adam

img_in = layers.Input(shape=t_x.shape[1:], name="RGB_Input")

c1 = layers.Conv2D(16, (3,3), padding='same', activation='relu')(img_in)
c2 = layers.Conv2D(32, (3,3), padding='same', activation='relu')(c1)
p1 = layers.MaxPooling2D((2,2))(c2)

c3 = layers.Conv2D(32, (3,3), padding='same', activation='relu')(p1)
c4 = layers.Conv2D(32, (3,3), padding='same', activation='relu')(c3)
p2 = layers.MaxPooling2D((2,2))(c4)

c5 = layers.Conv2D(32, (3,3), padding='same', activation='relu')(p2)
c6 = layers.Conv2D(64, (3,3), padding='same', activation='relu')(c5)
p3 = layers.MaxPooling2D((2,2))(c6)

p5 = layers.GlobalMaxPooling2D()(p3)
dense1 = layers.Dense(128, activation='relu')(p5)
drop = layers.Dropout(0.5)(dense1)
out_layer = layers.Dense(1, activation='sigmoid')(drop)

ship_model = models.Model(inputs = [img_in], outputs = [out_layer], name = 'full_model')

ship_model.compile(optimizer = Adam(lr=LEARN_RATE), 
                   loss = 'binary_crossentropy',
                   metrics = ['binary_accuracy'])

ship_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RGB_Input (InputLayer)       (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 512, 512, 16)      448       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 512, 512, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 128, 128, 32)      0         
__________

In [33]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights_classification.best.hdf5".format('boat_detector')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [ ]:
train_gen.batch_size = BATCH_SIZE
import tensorflow as tf


ship_model.fit_generator(train_gen, 
                      validation_data = (valid_x, valid_y), 
                      epochs = 10,
                      callbacks = callbacks_list,
                      workers = 1)

Epoch 1/10


    1/15000 [..............................] - ETA: 16:25:17 - loss: 1.1865 - binary_accuracy: 0.5000

    2/15000 [..............................] - ETA: 12:07:43 - loss: 0.5934 - binary_accuracy: 0.7500

    3/15000 [..............................] - ETA: 10:14:20 - loss: 1.3061 - binary_accuracy: 0.6667

    4/15000 [..............................] - ETA: 9:00:42 - loss: 5.0091 - binary_accuracy: 0.5000 

...

   32/15000 [..............................] - ETA: 5:45:17 - loss: 2.7563 - binary_accuracy: 0.5469

   33/15000 [..............................] - ETA: 5:44:07 - loss: 2.7061 - binary_accuracy: 0.5455

   34/15000 [..............................] - ETA: 5:43:36 - loss: 2.6524 - binary_accuracy: 0.5441

   35/15000 [..............................] - ETA: 5:44:41 - loss: 2.6177 - binary_accuracy: 0.5429

   36/15000 [..............................] - ETA: 5:45:15 - loss: 2.5525 - binary_accuracy: 0.5556

In [ ]:
ship_model.load_weights(weight_path)
ship_model.save('full_ship_classification_model.h5')

# Run the test data
We use the sample_submission file as the basis for loading and running the images.

test_paths = os.listdir(test_image_dir)
print(len(test_paths), 'test images found')
submission_df = pd.read_csv('../input/sample_submission.csv')
submission_df['path'] = submission_df['ImageId'].map(lambda x: os.path.join(test_image_dir, x))

# Setup Test Data Generator
We use the same generator as before to read and preprocess images

test_gen = flow_from_dataframe(valid_idg, 
                               submission_df, 
                             path_col = 'path',
                            y_col = 'ImageId', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE, 
                              shuffle = False)

fig, m_axs = plt.subplots(3, 2, figsize = (20, 30))
for (ax1, ax2), (t_x, c_img_names) in zip(m_axs, test_gen):
    t_y = ship_model.predict(t_x)
    t_stack = ((t_x-t_x.min())/(t_x.max()-t_x.min()))[:, :, :, ::RGB_FLIP]
    ax1.imshow(montage_rgb(t_stack))
    ax1.set_title('images')
    alpha_stack = np.tile(np.expand_dims(np.expand_dims(t_y, -1), -1), [1, t_stack.shape[1], t_stack.shape[2], 1])
    rgba_stack = np.concatenate([t_stack, alpha_stack], -1)
    ax2.imshow(montage_rgb(rgba_stack))
    ax2.set_title('ships')
fig.savefig('test_predictions.png')

# Prepare Submission
Process all images (batchwise) and keep the score at the end

BATCH_SIZE = BATCH_SIZE*2 # we can use larger batches for inference
test_gen = flow_from_dataframe(valid_idg, 
                               submission_df, 
                             path_col = 'path',
                            y_col = 'ImageId', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = BATCH_SIZE, 
                              shuffle = False)

from tqdm import tqdm_notebook
all_scores = dict()
for _, (t_x, t_names) in zip(tqdm_notebook(range(test_gen.n//BATCH_SIZE+1)),
                            test_gen):
    t_y = ship_model.predict(t_x)[:, 0]
    for c_id, c_score in zip(t_names, t_y):
        all_scores[c_id] = c_score

# Show the Scores
Here we see the scores and we have to decide about a cut-off for counting an image as ship or not. We can be lazy and pick 0.5 but some more rigorous cross-validation would definitely improve this process.

submission_df['score'] = submission_df['ImageId'].map(lambda x: all_scores.get(x, 0))
submission_df['score'].hist()

# Make the RLE data if there is a ship
Here we make the RLE data for a positive image (assume every pixel is ship)

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
# empty image
zp_dim = 10
out_img = np.ones((768-2*zp_dim, 768-2*zp_dim), dtype=bool)
out_img = np.pad(out_img, ((zp_dim, zp_dim),), mode='constant', constant_values=0)
plt.matshow(out_img)
print(out_img.shape)
pos_ship_str = rle_encode(out_img)
print(pos_ship_str[:50])

# add the whole image if it is above the threshold
submission_df['EncodedPixels'] = submission_df['score'].map(lambda x: pos_ship_str if x>0.5 else None)

out_df = submission_df[['ImageId', 'EncodedPixels']]
out_df.to_csv('submission.csv', index=False)
out_df.head(20)